# DMSP-OLS intercalibration (10 min)

As described in {doc}`mod1_1_introduction_to_nighttime_light_data`, the DMSP-OLS sensors lacked on-board calibration, so there are noticeable variations from satellite to satellite. In order to adequately compare nighttime lights data across satellites, they must be calibrated. 

In this exercise, we're going to calibrate DMSP-OLS data based on a method established by Elvidge et al. (2009) {cite}`elvidge2009fifteen`.

**Prerequisites:**
- Make sure you have Python, Jupyter notebooks, and `ee` and `geemap` installed and are familiar with these packages
- If not, you'll want to review: 
    - {doc}`mod1_3_getting_started_with_Python`
    - {doc}`mod1_4_introduction_to_Jupyter_notebooks`
    - {doc}`mod1_5_introduction_to_GEE`
- You may also want to try this exercise to get familiar with GEE and the Python `geemap` and `ee` libraries:
    - {doc}`mod1_7_practical_exercise-image_visualization`

**Our tasks in this exercise:**
1. Reviewing the calibration coefficients and formula used by Elvidge
2. Apply calibration steps to a single image, 1996
3. Compare the differences between calibrated and non-calibrated images with a histogram.
4. Create a function that dynamically applies the calibration depending on a given satellite/year input.
5. Compare the difference in calibrated and non-calibrated time series.

## Reviewing the calibration coefficients and formula

The initial paper {cite}`elvidge2009fifteen` cited outlines the steps the authors took, if you're interested in the details. What's important for our purposes here, is that they derived a set of coefficients that, when applied to each satellite in the DMSP series, provides a more stable calibrated measure of nighttime lights that corrects for the sensor variation and allows us to more accurately compare data.

These coefficients map to the formula:
$$X' = C_{0} + C_{1}*X + C_{2}*X^{2}$$

Where:
- X: the input image, represented as a 2-dimensional matrix (recall these images are panchromatic so there is only one channel of light)
- $C_{0}, C_{1}, C_{2}$: the calibration coefficients that are assigned to each satellite
- X': the calibrated image

The initial paper was written in 2009, prior to the final year of DMSP-OLS. We can get a more recent series of coefficients (through 2012) here (using the same method):

```{figure} img/mod2-2-intercalib_coef.png
---
name: intercalib_coefficients
---
DMSP-OLS intercalibration {cite}`jiang2017assessing`
```

**Note:** this method is fine for purposes of this tutorial, but it's worth mentioning that there are several other methods to conduct this calibration as well.


## Apply calibration to a single image, 1996

### Initialize object and visualization

As previous tutorials, we'll initialize a `geemap` object, get the DMSP-OLS 1996 composite and visualize it.

For this exercise, we'll use the "stable_lights" band.

In [1]:
# import geemap and ee for our Python session
import geemap, ee

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

# set our initial map parameters for Washington, DC
center_lat = 38.9072
center_lon = -77.0369
zoomlevel=10

# initialize our map
dmspMap = geemap.Map(center=[center_lat,center_lon], zoom=zoomlevel)
dmspMap.add_basemap('SATELLITE')

# get 1996 composite, apply mask, and add as layer
dmsp1996 = ee.Image("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F101992").select('stable_lights')
dmspMap.addLayer(dmsp1996.mask(dmsp1996), {}, "DMSP-OLS 1996", opacity=0.75)

In [2]:
# display our map
dmspMap

Map(center=[38.9072, -77.0369], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

For 1996, there is only one satellite, F12, so we can reference the appropriate coefficients for F121996 from our table above:

- $C_{0}$ = -0.0959
- $C_{1}$ = 1.2727
- $C_{2}$ = -0.0040

GEE has an `.expression()` method which allows us to plug our formula in with these coefficients and get an adjusted image.

We add our coefficients to the appropriate terms of the polynomial and set our input image as the X variable.

In [3]:
dmsp1996_clbr = dmsp1996.expression('-0.0959 + (1.2727 * X) + (-0.0040 * X * X)',{'X':dmsp1996.select('stable_lights')})

This calibration adjusts the value of the Digital Number (DN)and in some cases, it will cause a pixel's value to go above 63. We also have pixels with DN's of less than 6. We'll cap the DNs at 63 and replace any values below 6 with 0s.

In [4]:
dmsp1996_clbr = dmsp1996_clbr.where(dmsp1996_clbr.gt(63),63).where(dmsp1996_clbr.lte(6),0)

Now let's add our layer and compare.

In [25]:
dmspMap.addLayer(dmsp1996_clbr.mask(dmsp1996_clbr), {}, "DMSP-OLS 1996 calibrated", opacity=0.75)

In [19]:
ee.Reducer.histogram()

EEException: Invalid argument specified for ee.List(): results

## References:
```{bibliography} ../references.bib
:filter: docname in docnames
```